In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import wandb
import tensorflow as tf
import numpy as np
import os
from prosit_t.wandb_agent.train_utils import get_proteometools_data
import pandas as pd
from dlomix.losses import masked_spectral_distance
from prosit_t.models.variable_seq_length_models import ProstTransformerDynamicLen
from prosit_t.data.parquet_to_tfdataset import get_tfdatasets
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
pd.set_option('mode.chained_assignment', None)

2023-10-11 12:40:29.087302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-11 12:40:30.010296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-p

In [3]:
tf.config.run_functions_eagerly(True)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
train_df = pd.read_parquet("/cmnfs/proj/prosit/Transformer/first_pool_train.parquet")

In [ ]:
len(train_df) / 1024

In [4]:
PROJECT_NAME = "transforming-prosit-first-pool"

In [ ]:
with wandb.init(project=PROJECT_NAME) as _:
    cb_wandb = WandbCallback()

    callback_earlystopping = EarlyStopping(
        monitor="val_loss",
        patience=8,
        min_delta=0.0001,
        restore_best_weights=True,
        verbose=1,
    )
    callback_reduce_lr = ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.8,
            patience=4,
    )
    callbacks = [cb_wandb, callback_earlystopping, callback_reduce_lr]
    
    train_data, val_data = get_tfdatasets(batch_size=1024)
    model = ProstTransformerDynamicLen(
        seq_length=30,
        embedding_output_dim=64,
        num_heads=16,
        num_transformers=3, 
        dense_dim_factor=4
    )
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(
        optimizer=optimizer,
        loss=masked_spectral_distance,
    )
    model.fit(
        train_data,
        validation_data=val_data,
        epochs=200
    )

wandb: Currently logged in as: mamisashvili-lizi (prosit-compms). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
2023-10-09 11:27:35.832205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43640 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6
/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/200


2023-10-09 11:31:56.437196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-09 11:31:57.631760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801


2534/2534 [==============================] - 509s 200ms/step - loss: 0.4521 - val_loss: 0.3260
Epoch 2/200
1839/2534 [====================>.........] - ETA: 2:11 - loss: 0.3235

In [8]:
model.summary()

Model: "prost_transformer_dynamic_len"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  0 (unused)
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 time_distributed (TimeDist  multiple                  390       
 ributed)                                                        
                                                                 
 string_lookup (StringLooku  multiple                  0         
 p)                                                              
                                                                 
 positional_embedding (Posi  multiple                  1408      
 tionalEmbedding)                                                
                                     